# Problem 4.3

## part a.


In [77]:
import pandas as pd, numpy as np, statsmodels.api as sm
from statsmodels.formula.api import glm

In [69]:
H = [1, 2, 3] # represents histologies I, II, III
S = [1, 2, 3] # represents disease stages 1, 2, 3
T = [1, 3, 5, 7, 9, 11, 13]# represents time intervals '0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12+']
                            # the 13 in particular might not be a good interval. I'd have to look at the underlying
                            # data to see what range the 12+ category covers


We have three categorical parameters with 3, 3, and 7 categories in each. There are also two response variables, expected number of deaths, $\mu_{ijk}$, and total time at risk, $t_{ijk}$. However, since in the model we use $\log{\left(\mu_{ijk}/t_{ijk}\right)}$, there is really only one response variable, $v_{ijk} = \log{\left(\mu_{ijk}/t_{ijk}\right)}$, that we can calculate for any points $i, j, k$. 

Since I'm a python user, and not R, I'm going to build the design matrix directly dataframe, from a csv file, from a spreadsheet.

In [70]:
with open("/opt/school/stat665/hw4/4_12_dataset.csv") as csv_in:
    df = pd.read_csv(csv_in, names = ['mu', 't', 'H', 'S', 'T'])

In [71]:
y = np.log(df['mu']/df['t'])
y[-20:] #display the last 20 values for analysis

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


43   -2.833213
44   -2.639057
45   -2.484907
46        -inf
47   -2.079442
48        -inf
49   -3.091042
50   -2.890372
51   -1.540445
52   -2.302585
53        -inf
54        -inf
55   -1.098612
56   -1.670682
57   -1.745239
58   -1.815290
59   -1.673976
60   -1.945910
61   -1.540445
62   -1.203973
dtype: float64

In [79]:
df["logMuT"] = np.log(df['mu']/df['t'])
df[-20:]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


,mu,t,H,S,T,logMuT
43,1,17,3,1,3,-2.833213
44,1,14,3,1,5,-2.639057
45,1,12,3,1,7,-2.484907
46,0,10,3,1,9,-inf
47,1,8,3,1,11,-2.079442
48,0,6,3,1,13,-inf
49,1,22,3,2,1,-3.091042
50,1,18,3,2,3,-2.890372
51,3,14,3,2,5,-1.540445
52,1,10,3,2,7,-2.302585


As we can see, the cells where $\mu$ is zero will not behave well in a log-linear model, and should be eliminated. This is done by using as our input matrix only those rows where $\mu != 0$.

In [78]:
lm = glm('logMuT ~ C(H) + C(S) + C(T)', df.loc[df['mu'] != 0], family=sm.families.Gaussian()).fit()
print(lm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 logMuT   No. Observations:                   58
Model:                            GLM   Df Residuals:                       47
Model Family:                Gaussian   Df Model:                           10
Link Function:               identity   Scale:                  0.271188015978
Method:                          IRLS   Log-Likelihood:                -38.357
Date:                Tue, 14 Nov 2017   Deviance:                       12.746
Time:                        19:28:27   Pearson chi2:                     12.7
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.3364      0.220    -15.143      0.000      -3.768      -2.905
C(H)[T.2]      0.2975      0.163      1.824      0.0

In [ ]:
The degrees of freedom are 52 in this table because there are 

In [83]:
lm = glm('logMuT ~ C(H) + C(S) + C(T) + C(S)*C(H)', df.loc[df['mu'] != 0], family=sm.families.Gaussian()).fit()
print(lm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 logMuT   No. Observations:                   58
Model:                            GLM   Df Residuals:                       43
Model Family:                Gaussian   Df Model:                           14
Link Function:               identity   Scale:                  0.264461835785
Method:                          IRLS   Log-Likelihood:                -35.049
Date:                Tue, 14 Nov 2017   Deviance:                       11.372
Time:                        19:42:08   Pearson chi2:                     11.4
No. Iterations:                     2                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.4209      0.252    -13.570      0.000      -3.915      -2.927
C(H)[T.2]               0

The degrees of freedom are lowered by four because there are now four additional additional parameters in the fitted model, corresponding with all combinations of 2 H and 2 S parameters.